# Learn Notebook

In [11]:
# Setup env
import datetime, rdflib, pdb, sys, hashlib, requests
import os, sys, rdflib, uuid
from bplustree import BPlusTree, StrSerializer
NOTEBOOK_DIR = os.path.split(os.path.abspath(os.curdir))[0]
PROJECT_DIR = os.path.split(NOTEBOOK_DIR)[0]
BASE_DIR = os.path.split(PROJECT_DIR)[0]
CACHE_DIR = os.path.join(PROJECT_DIR, "cache")
sys.path.append(CACHE_DIR)
import btree
print(datetime.datetime.now(), sys.platform, os.path.os.name, BASE_DIR, PROJECT_DIR)

2018-05-25 14:43:15.630370 win32 nt D:\2018 D:\2018\linked-data-fragments


In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

## Odds and Ends

###  When submiting a HTTP GET request to the Linked Data Fragments Server, do not use raw URL
Encoded the raw URL value using `urllib.parse.urlencode` method. 

In [18]:
print(rdflib.RDF.type)
uncoded_url = "http://localhost:7000/?p={}".format(rdflib.RDF.type)
result = requests.get(uncoded_url)
print(uncoded_url, result.status_code)
print(result.json().get("metadata"))
import urllib.parse
print("http://localhost:7000/{}".format(urllib.parse.urlencode({"p": str(rdflib.RDF.type)})))

http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://localhost:7000/?p=http://www.w3.org/1999/02/22-rdf-syntax-ns#type 200
{'subject': 'http://localhost:7000/4779d612-605d-11e8-a669-001fc69cf6df', 'predicate': 'void:triples', 'object': 0}
http://localhost:7000/p=http%3A%2F%2Fwww.w3.org%2F1999%2F02%2F22-rdf-syntax-ns%23type


In [13]:
result = requests.get("http://localhost:7000/?p=http%3A%2F%2Fwww.w3.org%2F1999%2F02%2F22-rdf-syntax-ns%23type")
print(result.json().get("metadata"))

{'subject': 'http://localhost:7000/a8fa9100-605c-11e8-81ee-001fc69cf6df', 'predicate': 'void:triples', 'object': 52}


So what I think is happening is that Falcon is stripping off characters the final **#** in the url when the unencoded URL is being used. 